In [25]:
# imports
import pandas as pd
import numpy as np
from typing import List
import os
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import warnings


In [45]:
behaviors = pd.read_csv("MIND/behaviors_testdataset.csv", sep=';', header=None)
behaviors.columns =['User', 'Time', 'ID', 'Impressions'] 
behaviors = behaviors.drop(['Time'], axis=1)
behaviors.head()

,User,ID1,ID
0,U13000,N42782 N18445 N49749,N7482-1 N6379-0
1,U14000,N9721 N61358 N11360 N64593 N9900 N46688,N1145-1 N34930-1 N53526-0 N3574-0 N751-0 N2202...
2,U15000,N42474 N41013 N45970 N4015 N56461 N11161 N31633,N4303-1 N58730-0 N64130-0 N38064-0 N49210-0 N92-0
3,U13740,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
4,U91836,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...


In [27]:
news = pd.read_csv("MIND/news_testdataset.csv") 
news.head()

,ID,Category,SubCategory,Content
0,N53526,health,voices,I Was An NBA Wife. Here's How It Affected My M...
1,N9721,health,nutrition,"50 Foods You Should Never Eat, According to He..."
2,N39758,health,nutrition,25 Biggest Grocery Store Mistakes Making You G...
3,N3574,autos,autosnews,Ford Bronco Test Mule Spotted Flexing Its Musc...
4,N42474,news,newsbusiness,Trump's Trustbusters Bring Microsoft Lessons t...


In [38]:
def get_user_related_content(user):
    user_row = behaviors[behaviors['User'] == user]
    
    if user_row.empty:
        return [("No data found for user", user)]
    
    user_ids = user_row['ID'].iloc[0].split()
    
    result = []
    
    for user_id in user_ids:
        id_row = news[news['ID'] == user_id]
        
        if not id_row.empty:
            content = id_row['Content'].iloc[0]
            result.append((user_id, content))
    
    return result

[N42782: Three takeaways from Yankees' ALCS Game 5 victory over the Astros The Yankees kept hope alive thanks to some impressive starting pitching and a pair of early home runs.] [N18445: Michigan sends breakup tweet to Notre Dame as series goes on hold Parting is such sweet sorrow, say the Wolverines.] [N49749: 'Unbelievable.' Chiefs fans stunned, mad after loss to the Titans What the heck happened? The Chiefs were lining up for a field goal that would have given them and eight-point lead late in Sunday's game against the Tennessee Titans in Nashville. The snap came before holder Dustin Colquitt was ready and he threw the ball away. Colquitt was called for intentional grounding and the Titans took over. They drove down the field for a touchdown, two-point ...] 

In [40]:
user_input = 'U13000'
output = get_user_related_content(user_input)

#print(f"User: {user_input}", end=" ")
for user_id, content in output:
    print(f"[{user_id}: {content}]", end=" ")

[N42782: Three takeaways from Yankees' ALCS Game 5 victory over the Astros The Yankees kept hope alive thanks to some impressive starting pitching and a pair of early home runs.] [N18445: Michigan sends breakup tweet to Notre Dame as series goes on hold Parting is such sweet sorrow, say the Wolverines.] [N49749: 'Unbelievable.' Chiefs fans stunned, mad after loss to the Titans What the heck happened? The Chiefs were lining up for a field goal that would have given them and eight-point lead late in Sunday's game against the Tennessee Titans in Nashville. The snap came before holder Dustin Colquitt was ready and he threw the ball away. Colquitt was called for intentional grounding and the Titans took over. They drove down the field for a touchdown, two-point ...] 

In [43]:
import random

def get_random_news(n):
    random_news = random.sample(list(news.itertuples(index=False, name=None)), n)
    return [(news[0], news[3]) for news in random_news]


In [44]:
n_random_news = 3  # Change this to the desired number of random news
random_news_output = get_random_news(n_random_news)

print("Random News:")
for news_id, news_content in random_news_output:
    print(f"[{news_id}: {news_content}]", end=" ")

Random News:
[N6695: 'He fought them,' father says of son killed on birthday defending his family in home Brenton Estorffe, an Australian living in Texas, was died defending his family.] [N43299: Tigers put All-Star P Matthew Boyd on trade block Coming off one of the worst seasons in franchise history, it sure looks like the Detroit Tigers are going to further blow things up.] [N21395: Drone video shows the devastated compound where al-Baghdadi died Drone video showed there was little left standing of the compound at Barisha, Syria, where ISIS leader Abu Bakr al-Baghdadi was killed.] 

For Impressions

In [50]:
behaviors = pd.read_csv("MIND/behaviors_testdataset.csv", sep=';', header=None)
behaviors.columns =['User', 'Time', 'Read', 'ID'] 
behaviors = behaviors.drop(['Time'], axis=1)
behaviors.head()

,User,Read,ID
0,U13000,N42782 N18445 N49749,N7482-1 N6379-0
1,U14000,N9721 N61358 N11360 N64593 N9900 N46688,N1145-1 N34930-1 N53526-0 N3574-0 N751-0 N2202...
2,U15000,N42474 N41013 N45970 N4015 N56461 N11161 N31633,N4303-1 N58730-0 N64130-0 N38064-0 N49210-0 N92-0
3,U13740,N55189 N42782 N34694 N45794 N18445 N63302 N104...,N55689-1 N35729-0
4,U91836,N31739 N6072 N63045 N23979 N35656 N43353 N8129...,N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...


In [53]:
def clean_id(id_value):
    # Remove index part ("-1" or "-0")
    return id_value.rstrip('01').rstrip('-')

def get_user_related_content(user):
    user_row = behaviors[behaviors['User'] == user]
    
    if user_row.empty:
        return [("No data found for user", user)]
    
    user_ids = user_row['ID'].iloc[0].split()
    
    result = []
    
    for user_id in user_ids:
        cleaned_id = clean_id(user_id)
        id_row = news[news['ID'] == cleaned_id]
        
        if not id_row.empty:
            content = id_row['Content'].iloc[0]
            result.append((cleaned_id, content))
    
    return result

# Example usage
user_input = 'U13000'
output = get_user_related_content(user_input)

if output:
    #print(f"User: {user_input}", end=" ")
    for user_id, content in output:
        print(f"[{user_id}: {content}]", end=" ")
else:
    print(f"No data found for user {user_input}")


[N7482: St. Dominic soccer player tries to kick cancer to the curb Sometimes, what happens on the sidelines can be even more important than what happens on the field. That was the case recently for the improving St. Dominic boys soccer team. "We're led by some great seniors and they're learning a lot," said head soccer coach Greg Koeller. Since 2000, the Crusaders have won five state titles. And if that wasn't enough motivation, they have players who provide inspiration. Like No. 2, Clay North. "Clay is just an...] [N6379: California cops respond to 2 homeless men shot with arrows: report Two homeless men living at an encampment near San Francisco were shot with arrows early Sunday in an attack that was not random, a report said.] 